# Central Differential Privacy (Client-Side Clipping)



Central Differential Privacy (Client-Side Clipping) is a setup where the clipping of model updates happens on each client before sending the updates to the central server. After collecting all the clipped updates, the server aggregates them and adds noise centrally to enforce differential privacy.

In this setup:

- Clipping happens locally on the client — helping reduce the influence of outlier updates early.

- Noise is added by the server — simplifying coordination of privacy guarantees.

The specific strategy used here is:

## CNN: DifferentialPrivacyClientSideFixedClipping

This strategy enforces a fixed clipping norm on all clients before updates are sent, ensuring consistency across client contributions and helping the server apply a uniform DP guarantee during aggregation.

<br><p>

----

In [2]:
%load_ext autoreload
%load_ext watermark
    
%autoreload 2
%watermark --python -p torch,flwr

Python implementation: CPython
Python version       : 3.12.10
IPython version      : 9.1.0

torch: 2.6.0
flwr : 1.18.0



--------

## Loading Dependencies

In [ ]:
import os
import sys
sys.path.append("../")

import ray
import time

import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights

from logging import ERROR

from flwr.common import ndarrays_to_parameters, Context 
from flwr.client import Client, ClientApp
from flwr.client.mod import fixedclipping_mod
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import DifferentialPrivacyClientSideFixedClipping, FedAvg
from flwr.simulation import run_simulation

from src.config import ExperimentName
from src.paths import RAY_LOG_DIR
from src.FL_client import MedicalImageClient
from src.FL_server import weighted_average, build_evaluate_fn
from src.local_utility import load_yaml_config, set_device, prepare_FL_dataset, get_weights

from src.tracker import reset_base_memory_csv

In [ ]:
# Replace with appropritate values

data_name_ = 'alzheimer' #"skin_lesions" #
base_type_ = "CNN"           
exp_name_ = "CDP-CF"
num_labels_ = 4
experiment_name_ = ExperimentName.FL_CDP_CF_CNN

In [ ]:
DEVICE = set_device()
reset_base_memory_csv() 

fed_config = load_yaml_config(key="experiments", item_name=experiment_name_)
client_dataloaders = prepare_FL_dataset(exp_name= exp_name_, data_name=data_name_, base_type=base_type_, augment_data=True)

In [ ]:
def client_fn(context: Context) -> Client:
    """
    Creates and initializes a federated learning client.

    This function initializes a client in the federated learning setup by 
    assigning a unique partitioned dataset and a machine learning model 
    for training and validation.

    Args:
        context (Context): The execution context containing client-specific configurations.

    Returns:
        Client: A configured federated learning client ready to participate in training.
    """
    partition_id = int(context.node_config["partition-id"]) #<--- Get the client partition ID
    
    model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_labels_)

    # Assign partitioned client dataset
    train_loader, val_loader, test_loader = client_dataloaders[partition_id]
    
    return MedicalImageClient(model, train_loader, val_loader, exp_name=exp_name_, data_name = data_name_, base_type=base_type_, client_id=partition_id).to_client()

client = ClientApp(client_fn, mods=[fixedclipping_mod])

In [ ]:
def server_fn(context: Context):
    """
    Creates and configures the federated learning server using the FedAvg strategy.

    This function initializes the federated learning server with a FedAvg strategy,
    specifying the parameters for client participation in training and evaluation,
    the global model evaluation function, and the metric aggregation function.

    Args:
        context (Context): The execution context for the federated learning server.

    Returns:
        ServerAppComponents: A configured server application with the defined strategy
        and server settings.
    """
    model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_labels_)
    params = ndarrays_to_parameters(get_weights(model))
    
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=1.0,                 #<--- Sample 100% of available clients for training
        fraction_evaluate=1.0,            #<--- Sample 100% of available clients for evaluation
        initial_parameters=params,        #<--- Initial model parameters
        evaluate_fn=build_evaluate_fn(    #<--- Global evaluation function
            exp_name = exp_name_, 
            base_type = base_type_,
            data_name=data_name_, 
            experiment_item=experiment_name_, 
            num_labels=num_labels_
            ),
        evaluate_metrics_aggregation_fn=weighted_average,  #<-- pass the metric aggregation function
    )
    
    # Wrap the strategy with the DifferentialPrivacyServerSideFixedClipping wrapper
    dp_strategy = DifferentialPrivacyClientSideFixedClipping(
        strategy= strategy,
        noise_multiplier= 0.0, #0.001,
        num_sampled_clients=fed_config.get("num_clients"),
        clipping_norm=500.5 #1e-4
        )
    
    # Configure the server with the specified number of federated rounds
    sever_config = ServerConfig(num_rounds=fed_config['num_rounds'])
    
    return ServerAppComponents(strategy = dp_strategy, config = sever_config)

# Wrap the server function in a ServerApp, and instantiate it
server = ServerApp(server_fn = server_fn) 

----- 

Set up our backend configurations before running the simulation

> - `init_args`: filter logging coming from the Simulation Engine so it's more readable in notebooks
> - `client_resources`: by default, allocate to each client 1x CPU and 0x GPUs. However, when running on GPU, assign an entire GPU for each client

In [7]:
backend_setup = {
    "init_args": {
        "logging_level": ERROR, 
        "log_to_driver": fed_config.get("log_to_driver")
    },
    "client_resources": {
        "num_cpus": fed_config.get("num_cpus"), 
        "num_gpus": fed_config.get("num_gpus")            
    },
}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda": 
    backend_setup["client_resources"] = {"num_cpus": 1, "num_gpus": 1.0}

--- 

## Initiate the Simulation 

Initiate the simulation by passing the server and client apps, and specify the number of supernodes that will be selected on every round. 

In [ ]:
project_root =  os.path.abspath("../..")

ray.shutdown()

ray.init(
    _temp_dir=str(RAY_LOG_DIR),
    runtime_env={
        "env_vars": {
            "PYTHONWARNINGS": "ignore::DeprecationWarning",  # More specific warning filter
            "OMP_NUM_THREADS": "1"  # Prevents thread oversubscription
        },
        "working_dir": project_root,
        'excludes': ['data', '.cache', '.docker', '.local', 'logs/model']
    },
    ignore_reinit_error=True
)

start_time = time.perf_counter()

run_simulation(
    server_app = server,
    client_app = client,
    num_supernodes = fed_config.get("num_clients"),
    backend_config=backend_setup
)


end_time = time.perf_counter()
duration = end_time - start_time
print(f"\n🕒 Total Time: {duration // 60:.0f} min {duration % 60:.0f} sec")

ray.shutdown()

2025-05-06 17:52:44,770	INFO worker.py:1771 -- Started a local Ray instance.
2025-05-06 17:52:45,258	INFO packaging.py:530 -- Creating a file package for local directory '/home/emeka/PrivacyBench'.
2025-05-06 17:52:45,818	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_34b941e43a8f6fc6.zip' (17.99MiB) to Ray cluster...
2025-05-06 17:52:45,874	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_34b941e43a8f6fc6.zip'.
INFO :      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
/home/emeka/.local/share/uv/python/cpython-3.12.10-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=95096) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()
INFO :      initial parameters (loss, other metrics): 

(ClientAppActor pid=115202) 
(ClientAppActor pid=115202) 🔎 Tracker: FL + CDP-CF (CNN)
(ClientAppActor pid=115202) 📁 Logs saved to: /home/emeka/PrivacyBench/logs/emissions/FL_CDP-CF_CNN/client_emissions.csv
(ClientAppActor pid=115202) ⏱️ Total training time: 1 minutes 41 seconds


(ClientAppActor pid=115202) INFO :      fixedclipping_mod: parameters are clipped by value: 500.5000.
(ClientAppActor pid=115202) 2025-05-06 17:54:40,808 - INFO - fixedclipping_mod: parameters are clipped by value: 500.5000.
(ClientAppActor pid=115202) Seed set to 42
(ClientAppActor pid=115202) /home/emeka/.local/share/uv/python/cpython-3.12.10-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=115202) is multi-threaded, use of fork() may lead to deadlocks in the child.
(ClientAppActor pid=115202)   self.pid = os.fork()


(ClientAppActor pid=115201) 


(raylet) [2025-05-06 17:54:44,767 E 113297 113332] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-06_17-52-42_619310_95096 is over 95% full, available space: 721178624; capacity: 52589998080. Object creation will fail if spilling is required.
(ClientAppActor pid=115201) INFO :      fixedclipping_mod: parameters are clipped by value: 500.5000.
(ClientAppActor pid=115201) 2025-05-06 17:54:42,850 - INFO - fixedclipping_mod: parameters are clipped by value: 500.5000.
(raylet) [2025-05-06 17:54:54,776 E 113297 113332] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-06_17-52-42_619310_95096 is over 95% full, available space: 721072128; capacity: 52589998080. Object creation will fail if spilling is required.
(raylet) [2025-05-06 17:55:04,785 E 113297 113332] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-06_17-52-42_619310_95096 is over 95% full, available space: 720797696; 

...

(ClientAppActor pid=115202) 


(ClientAppActor pid=115202) INFO :      fixedclipping_mod: parameters are clipped by value: 500.5000.
(ClientAppActor pid=115202) 2025-05-06 18:07:46,166 - INFO - fixedclipping_mod: parameters are clipped by value: 500.5000.
(raylet) [2025-05-06 18:07:55,530 E 113297 113332] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-06_17-52-42_619310_95096 is over 95% full, available space: 711888896; capacity: 52589998080. Object creation will fail if spilling is required.
(raylet) [2025-05-06 18:08:05,541 E 113297 113332] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-06_17-52-42_619310_95096 is over 95% full, available space: 711536640; capacity: 52589998080. Object creation will fail if spilling is required.
(raylet) [2025-05-06 18:08:15,549 E 113297 113332] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-06_17-52-42_619310_95096 is over 95% full, available space: 711483392; 

(ClientAppActor pid=115201) 
(ClientAppActor pid=115201) 🔎 Tracker: FL + CDP-CF (CNN) [repeated 2x across cluster]
(ClientAppActor pid=115201) 📁 Logs saved to: /home/emeka/PrivacyBench/logs/emissions/FL_CDP-CF_CNN/client_emissions.csv [repeated 2x across cluster]
(ClientAppActor pid=115201) ⏱️ Total training time: 1 minutes 28 seconds [repeated 2x across cluster]


(ClientAppActor pid=115201) INFO :      fixedclipping_mod: parameters are clipped by value: 500.5000.
(ClientAppActor pid=115201) 2025-05-06 18:09:12,052 - INFO - fixedclipping_mod: parameters are clipped by value: 500.5000.
INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      aggregate_fit: central DP noise with 0.0000 stdev added
/home/emeka/.local/share/uv/python/cpython-3.12.10-linux-x86_64-gnu/lib/python3.12/multiprocessing/popen_fork.py:66: DeprecationWarning: This process (pid=95096) is multi-threaded, use of fork() may lead to deadlocks in the child.
  self.pid = os.fork()
(raylet) [2025-05-06 18:09:15,604 E 113297 113332] (raylet) file_system_monitor.cc:111: /home/emeka/PrivacyBench/logs/ray/session_2025-05-06_17-52-42_619310_95096 is over 95% full, available space: 710791168; capacity: 52589998080. Object creation will fail if spilling is required.
INFO :      fit progress: (5, 0.2680968377739191, {'accuracy': 0.91015625}, 985.0255587970005)
INFO :      co


🕒 Total Time: 16 min 35 sec
